# ⚽ PID Controller Ball Chase

The red ball uses a PID (Proportional-Integral-Derivative) Controller to chase the blue ball moving in a circle.

Adjust the PID gains to see how different control strategies affect the chase behavior. PID is simpler but requires tuning!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Circle
from IPython.display import HTML
from ipywidgets import interact, FloatSlider, Button, VBox, HBox, Output, IntSlider
import ipywidgets as widgets

# Use inline backend for static plots, but we'll create HTML5 animation
%matplotlib inline
plt.rcParams['animation.html'] = 'jshtml'

## PID Controller Implementation

In [ ]:
class PIDController:
    """PID Controller for tracking control"""
    
    def __init__(self, kp=0.1, ki=0.02, kd=0.15):
        self.kp = kp  # Proportional gain
        self.ki = ki  # Integral gain
        self.kd = kd  # Derivative gain
        
        self.integral = 0
        self.prev_error = 0
        self.dt = 0.016  # ~60fps
        
        # Store individual terms for visualization
        self.p_term = 0
        self.i_term = 0
        self.d_term = 0
    
    def compute(self, error):
        """Compute PID control output"""
        # Proportional term
        self.p_term = self.kp * error
        
        # Integral term (with anti-windup)
        self.integral += error * self.dt
        self.integral = np.clip(self.integral, -1000, 1000)
        self.i_term = self.ki * self.integral
        
        # Derivative term
        self.d_term = self.kd * (error - self.prev_error) / self.dt
        
        # Total output
        output = self.p_term + self.i_term + self.d_term
        
        # Update for next iteration
        self.prev_error = error
        
        return output
    
    def reset(self):
        """Reset controller state"""
        self.integral = 0
        self.prev_error = 0
        self.p_term = 0
        self.i_term = 0
        self.d_term = 0
    
    def set_gains(self, kp, ki, kd):
        """Update PID gains"""
        self.kp = kp
        self.ki = ki
        self.kd = kd

## Ball Chase Simulation with Predictive Steering

In [ ]:
class PIDSimulation:
    """Simulation of red ball chasing blue ball using predictive steering"""
    
    def __init__(self, center=(350, 350), radius=120):
        self.center = np.array(center, dtype=float)
        self.radius = radius
        
        # Blue ball (target) - moves in circle
        self.blue_angle = 0
        self.blue_speed = 0.008  # radians per frame
        self.blue_radius = 12
        self.blue_linear_speed = self.radius * self.blue_speed
        
        # Red ball (hunter) - uses predictive steering
        self.red_pos = np.array([self.center[0] - self.radius, self.center[1]], dtype=float)
        self.red_radius = 12
        self.red_linear_speed = self.blue_linear_speed  # Match blue ball speed
        
        # Tracking
        self.caught = False
        self.catch_distance = 25
        
        # Trails
        self.blue_trail = []
        self.red_trail = []
        self.max_trail = 200
        
        # Current positions
        self.blue_pos = self.center + np.array([self.radius, 0])
    
    def update(self):
        """Update simulation one frame"""
        # Update blue ball position (circular motion)
        self.blue_angle += self.blue_speed
        self.blue_pos = self.center + self.radius * np.array([
            np.cos(self.blue_angle),
            np.sin(self.blue_angle)
        ])
        
        # Predict where blue ball will be (several frames ahead)
        predict_frames = 10
        future_angle = self.blue_angle + self.blue_speed * predict_frames
        future_blue_pos = self.center + self.radius * np.array([
            np.cos(future_angle),
            np.sin(future_angle)
        ])
        
        # Calculate direction from red ball to predicted blue ball position
        direction = future_blue_pos - self.red_pos
        distance = np.linalg.norm(direction)
        
        # Normalize to get unit direction vector
        if distance > 0:
            direction = direction / distance
        
        # Red ball moves at constant linear speed in the direction of target
        self.red_pos += direction * self.blue_linear_speed
        
        # Check if caught (using actual current position)
        catch_dist = np.linalg.norm(self.blue_pos - self.red_pos)
        self.caught = catch_dist < self.catch_distance
        
        # Update trails
        self.blue_trail.append(self.blue_pos.copy())
        self.red_trail.append(self.red_pos.copy())
        
        if len(self.blue_trail) > self.max_trail:
            self.blue_trail.pop(0)
            self.red_trail.pop(0)
        
        return catch_dist
    
    def reset(self):
        """Reset simulation to initial state"""
        self.blue_angle = 0
        self.red_pos = np.array([self.center[0] - self.radius, self.center[1]], dtype=float)
        self.caught = False
        self.blue_trail = []
        self.red_trail = []
    
    def set_radius(self, radius):
        """Update circle radius"""
        self.radius = radius
        self.blue_linear_speed = self.radius * self.blue_speed
        self.red_linear_speed = self.blue_linear_speed
        self.reset()

## Create and Display Animation

This will create an interactive HTML5 animation with JavaScript playback controls.

In [ ]:
# Create simulation
sim = PIDSimulation(center=(350, 350), radius=120)

# Create figure with dark background
fig, ax = plt.subplots(figsize=(10, 10))
ax.set_xlim(0, 700)
ax.set_ylim(0, 700)
ax.set_aspect('equal')
ax.set_facecolor('#1a1a2e')
fig.patch.set_facecolor('#1a1a2e')
ax.axis('off')

# Draw circular path
circle = Circle(sim.center, sim.radius, fill=False, edgecolor='white', 
                linestyle='--', linewidth=2, alpha=0.3)
ax.add_patch(circle)

# Initialize plot elements
blue_ball = Circle(sim.blue_pos, sim.blue_radius, color='#3498db', zorder=10)
red_ball = Circle(sim.red_pos, sim.red_radius, color='#e74c3c', zorder=10)

ax.add_patch(blue_ball)
ax.add_patch(red_ball)

blue_trail_line, = ax.plot([], [], color='#3498db', alpha=0.3, linewidth=1.5)
red_trail_line, = ax.plot([], [], color='#e74c3c', alpha=0.3, linewidth=1.5)

status_text = ax.text(350, 650, '🎯 Chasing...', ha='center', va='top', 
                     fontsize=16, color='white', weight='bold',
                     bbox=dict(boxstyle='round', facecolor='#e74c3c', alpha=0.8, pad=0.5))

info_text = ax.text(350, 50, '', ha='center', va='bottom',
                   fontsize=10, color='white', family='monospace')

def animate(frame):
    """Animation function called for each frame"""
    # Update simulation
    distance = sim.update()
    
    # Update ball positions
    blue_ball.center = sim.blue_pos
    red_ball.center = sim.red_pos
    
    # Update trails
    if len(sim.blue_trail) > 1:
        blue_trail = np.array(sim.blue_trail)
        blue_trail_line.set_data(blue_trail[:, 0], blue_trail[:, 1])
        
        red_trail = np.array(sim.red_trail)
        red_trail_line.set_data(red_trail[:, 0], red_trail[:, 1])
    
    # Update status
    if sim.caught:
        status_text.set_text('🎉 Caught! ✨')
        status_text.set_bbox(dict(boxstyle='round', facecolor='#ff9800', alpha=0.8, pad=0.5))
    else:
        status_text.set_text('🎯 Chasing...')
        status_text.set_bbox(dict(boxstyle='round', facecolor='#e74c3c', alpha=0.8, pad=0.5))
    
    # Update info
    info_text.set_text(f'Frame: {frame} | Distance: {distance:.1f} px | '
                      f'Blue: ({sim.blue_pos[0]:.1f}, {sim.blue_pos[1]:.1f}) | '
                      f'Red: ({sim.red_pos[0]:.1f}, {sim.red_pos[1]:.1f})')
    
    return blue_ball, red_ball, blue_trail_line, red_trail_line, status_text, info_text

# Create animation (500 frames at 50ms interval = 25 seconds)
anim = FuncAnimation(fig, animate, frames=500, interval=50, blit=True, repeat=True)

# Display as HTML5 video with controls
plt.close()  # Don't show the static figure
HTML(anim.to_jshtml())

## Interactive Parameter Controls

Use the slider below to experiment with different circle radius values.

In [ ]:
def create_animation_with_params(radius=120, num_frames=500):
    """Create animation with specified parameters"""
    # Create new simulation
    sim = PIDSimulation(center=(350, 350), radius=radius)
    
    # Create figure
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.set_xlim(0, 700)
    ax.set_ylim(0, 700)
    ax.set_aspect('equal')
    ax.set_facecolor('#1a1a2e')
    fig.patch.set_facecolor('#1a1a2e')
    ax.axis('off')
    
    # Draw circular path
    circle = Circle(sim.center, sim.radius, fill=False, edgecolor='white', 
                    linestyle='--', linewidth=2, alpha=0.3)
    ax.add_patch(circle)
    
    # Initialize plot elements
    blue_ball = Circle(sim.blue_pos, sim.blue_radius, color='#3498db', zorder=10)
    red_ball = Circle(sim.red_pos, sim.red_radius, color='#e74c3c', zorder=10)
    
    ax.add_patch(blue_ball)
    ax.add_patch(red_ball)
    
    blue_trail_line, = ax.plot([], [], color='#3498db', alpha=0.3, linewidth=1.5)
    red_trail_line, = ax.plot([], [], color='#e74c3c', alpha=0.3, linewidth=1.5)
    
    status_text = ax.text(350, 650, '🎯 Chasing...', ha='center', va='top', 
                         fontsize=16, color='white', weight='bold',
                         bbox=dict(boxstyle='round', facecolor='#e74c3c', alpha=0.8, pad=0.5))
    
    param_text = ax.text(350, 620, f'Radius: {radius} px | Speed: {sim.blue_linear_speed:.2f} px/frame', 
                        ha='center', va='top', fontsize=12, color='white')
    
    def animate(frame):
        distance = sim.update()
        
        blue_ball.center = sim.blue_pos
        red_ball.center = sim.red_pos
        
        if len(sim.blue_trail) > 1:
            blue_trail = np.array(sim.blue_trail)
            blue_trail_line.set_data(blue_trail[:, 0], blue_trail[:, 1])
            red_trail = np.array(sim.red_trail)
            red_trail_line.set_data(red_trail[:, 0], red_trail[:, 1])
        
        if sim.caught:
            status_text.set_text('🎉 Caught! ✨')
            status_text.set_bbox(dict(boxstyle='round', facecolor='#ff9800', alpha=0.8, pad=0.5))
        else:
            status_text.set_text('🎯 Chasing...')
            status_text.set_bbox(dict(boxstyle='round', facecolor='#e74c3c', alpha=0.8, pad=0.5))
        
        return blue_ball, red_ball, blue_trail_line, red_trail_line, status_text
    
    anim = FuncAnimation(fig, animate, frames=num_frames, interval=50, blit=True, repeat=True)
    plt.close()
    return HTML(anim.to_jshtml())

# Interactive widget
interact(create_animation_with_params,
         radius=IntSlider(value=120, min=50, max=200, step=10, description='Radius:'),
         num_frames=IntSlider(value=300, min=100, max=1000, step=100, description='Frames:'))

## 📐 PID Controller: Mathematical Foundation

### Intuitive Understanding

A PID controller is like a person trying to catch a moving ball. They adjust their movement based on:

**"Where is the ball now? (P) How far behind am I catching up? (I) Is it moving towards or away from me? (D)"**

The controller combines three "feedback" signals to decide how hard to steer.

### The Complete PID Control Law

$$u(t) = K_p \cdot e(t) + K_i \cdot \int_0^t e(\tau) d\tau + K_d \cdot \frac{de(t)}{dt}$$

Where:
- $u(t)$ = control output (what we do)
- $e(t) = \text{setpoint} - \text{measured value}$ = error
- $K_p$ = proportional gain
- $K_i$ = integral gain  
- $K_d$ = derivative gain

### The Three Components

#### 1. Proportional Term (P)

**Simple reaction:** "How far away are you? Act proportionally."

$$u_P(t) = K_p \cdot e(t)$$

- If error is large, apply large control signal
- If error is small, apply small control signal
- Problem: Cannot reach zero error (steady-state error remains)

#### 2. Integral Term (I)

**Historical correction:** "Have you been far away for a long time? Keep pushing harder."

$$u_I(t) = K_i \cdot \int_0^t e(\tau) d\tau$$

Discrete implementation:
$$I_{\text{sum}} \leftarrow I_{\text{sum}} + e(k) \cdot \Delta t$$
$$u_I(k) = K_i \cdot I_{\text{sum}}$$

- Accumulates error over time
- Eliminates steady-state error
- Can cause overshoot if too high

#### 3. Derivative Term (D)

**Predictive braking:** "Are you getting closer or further away? Adjust your steering rate."

$$u_D(t) = K_d \cdot \frac{de(t)}{dt}$$

Discrete implementation:
$$\Delta e = e(k) - e(k-1)$$
$$u_D(k) = K_d \cdot \frac{\Delta e}{\Delta t}$$

- Responds to rate of change of error
- Reduces overshoot
- Adds damping (smoothing)

### Application to 2D Ball Chasing

We apply PID control to both X and Y coordinates independently:

**For X-direction:**
$$e_x = x_{\text{blue}} - x_{\text{red}}$$
$$u_x = K_p \cdot e_x + K_i \cdot \int e_x dt + K_d \cdot \frac{de_x}{dt}$$
$$x_{\text{red,new}} = x_{\text{red}} + u_x$$

**For Y-direction:**
$$e_y = y_{\text{blue}} - y_{\text{red}}$$
$$u_y = K_p \cdot e_y + K_i \cdot \int e_y dt + K_d \cdot \frac{de_y}{dt}$$
$$y_{\text{red,new}} = y_{\text{red}} + u_y$$

### PID Tuning Guidelines

| Parameter | Effect of Increasing | Danger Zone |
|-----------|---------------------|-------------|
| **Kp** | Faster response to error, more aggressive | Too high → oscillation, overshoot |
| **Ki** | Eliminates steady-state error, maintains effort | Too high → integral windup, big overshoot |
| **Kd** | Reduces overshoot, smooths response | Too high → noisy, jerky, sensitive to noise |

### Advantages of PID Control

- **Simplicity:** Just three numbers to tune
- **Universality:** Works for almost any system
- **No Model Needed:** Doesn't require knowing system dynamics
- **Robustness:** Works well even with unknown disturbances
- **Low Computational Cost:** O(1) per iteration
- **Proven in Industry:** 70+ years of real-world success

### When to Use PID vs Kalman

**Use PID When:**
- You need simple, fast control
- System is well-behaved and easily controllable
- Measurements are clean (not too noisy)
- You have computational constraints

**Use Kalman When:**
- You have very noisy measurements
- You want to estimate hidden states
- System dynamics are well-understood and linear
- You want mathematical optimality guarantees

**Use Combined (PID + Kalman):**
- Use Kalman to estimate true state from noisy sensors
- Use PID to control based on the estimated state
- Best of both worlds!